In [1]:
import pandas as pd
import requests

# Load your CSV file
df = pd.read_csv('MSCityData.csv')
pantry_df = pd.read_csv('MSPantryLatLong.csv')

# Function to get county name from latitude and longitude
def get_county(lat, long, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={lat},{long}&key={api_key}"
    response = requests.get(url)
    if response.status_code == 200:
        results = response.json()['results']
        for result in results:
            for component in result['address_components']:
                if 'administrative_area_level_2' in component['types']:
                    return component['long_name']
    return None

# Replace 'YOUR_API_KEY' with your actual Google Maps Geocoding API key
api_key = 'YOUR_API_KEY'

# Apply the function to each row
#df['County'] = df.apply(lambda row: get_county(row['lat'], row['lng'], api_key), axis=1)

# Save the updated DataFrame to a new CSV file
#df.to_csv('updated_MSCityData.csv', index=False)

/Users/CameronField/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [ ]:
# Apply the function to each row
#pantry_df['County'] = df.apply(lambda row: get_county(row['Latitude'], row['Longitude'], api_key), axis=1)
#antry_df.to_csv('updated_MSPantryLatLong.csv', index=False)

In [15]:
import pandas as pd
import numpy as np
city_data = pd.read_csv('updated_MSCityData.csv')
city_data.head()

,pop2023,pop2020,growth,name,aland_sqmi,intptlat,intptlong,slug,usps,lng,lat,rank,density,County
0,143776,152846,-0.02019,Jackson,111.086,32.3158,-90.2128,jackson-ms,MS,-90.2128,32.3158,1,1294,Hinds County
1,70663,72837,-0.01004,Gulfport,55.629,30.4195,-89.0690,gulfport-ms,MS,-89.0690,30.4195,2,1270,Harrison County
2,56763,54774,0.01196,Southaven,41.325,34.9510,-89.9777,southaven-ms,MS,-89.9777,34.9510,3,1374,DeSoto County
3,48814,49456,-0.00434,Biloxi,42.987,30.4267,-88.9358,biloxi-ms,MS,-88.9358,30.4267,4,1136,Harrison County
4,46657,47275,-0.00437,Hattiesburg,53.432,31.3073,-89.3174,hattiesburg-ms,MS,-89.3174,31.3073,5,873,Forrest County


In [16]:
# Replace 'County' with an empty string in each cell of the 'County' column
city_data['County'] = city_data['County'].str.replace(' County', '')
city_data.head()


,pop2023,pop2020,growth,name,aland_sqmi,intptlat,intptlong,slug,usps,lng,lat,rank,density,County
0,143776,152846,-0.02019,Jackson,111.086,32.3158,-90.2128,jackson-ms,MS,-90.2128,32.3158,1,1294,Hinds
1,70663,72837,-0.01004,Gulfport,55.629,30.4195,-89.0690,gulfport-ms,MS,-89.0690,30.4195,2,1270,Harrison
2,56763,54774,0.01196,Southaven,41.325,34.9510,-89.9777,southaven-ms,MS,-89.9777,34.9510,3,1374,DeSoto
3,48814,49456,-0.00434,Biloxi,42.987,30.4267,-88.9358,biloxi-ms,MS,-88.9358,30.4267,4,1136,Harrison
4,46657,47275,-0.00437,Hattiesburg,53.432,31.3073,-89.3174,hattiesburg-ms,MS,-89.3174,31.3073,5,873,Forrest


In [17]:
county_df = pd.read_csv("CountyFIPopulation.csv")
# Ensure 'FI Population' is in a numeric format
county_df['FI Population'] = pd.to_numeric(county_df['FI Population'].str.replace('[^0-9.]', '', regex=True), errors='coerce')
county_df.head()

,County,FI Rate,FI Population
0,Adams,18.50%,5530
1,Alcorn,17.90%,6260
2,Amite,21.70%,2770
3,Attala,18.40%,3310
4,Benton,15.20%,1170


In [18]:
# Calculate total population of each county in 2020
total_county_population = city_data.groupby('County')['pop2020'].sum().reset_index()
total_county_population.rename(columns={'pop2020': 'TotalCountyPopulation'}, inplace=True)
total_county_population.head()

,County,TotalCountyPopulation
0,Adams,14202
1,Alcorn,17427
2,Amite,1722
3,Attala,7666
4,Benton,1321


In [19]:
# Merge this data back to the city_data DataFrame
city_data = pd.merge(city_data, total_county_population, on='County')
city_data.head()

,pop2023,pop2020,growth,name,aland_sqmi,intptlat,intptlong,slug,usps,lng,lat,rank,density,County,TotalCountyPopulation
0,143776,152846,-0.02019,Jackson,111.086,32.3158,-90.2128,jackson-ms,MS,-90.2128,32.3158,1,1294,Hinds,198680
1,26558,27909,-0.01641,Clinton,41.872,32.3544,-90.3419,clinton-ms,MS,-90.3419,32.3544,14,634,Hinds,198680
2,12144,12615,-0.01260,Byram,18.364,32.1903,-90.2860,byram-ms,MS,-90.2860,32.1903,32,661,Hinds,198680
3,1819,1876,-0.01034,Raymond,2.999,32.2609,-90.4083,raymond-ms,MS,-90.4083,32.2609,105,607,Hinds,198680
4,1481,1312,0.04149,Terry,3.813,32.1061,-90.2926,terry-ms,MS,-90.2926,32.1061,122,388,Hinds,198680


In [20]:
# Calculate the weight for each city
city_data['CityWeight'] = city_data['pop2020'] / city_data['TotalCountyPopulation']


# Merge the food insecure population data into the city_data DataFrame
combined_df = pd.merge(city_data, county_df[['County', 'FI Population']], on='County')
combined_df.head()

# Selecting only 'name' and 'CityWeight' columns, renaming 'name' to 'City'
city_weights = combined_df[['name', 'CityWeight']].rename(columns={'name': 'City'})
city_weights.head()

# Saving the resulting DataFrame to a CSV file
city_weights.to_csv('City_Weight.csv', index=False)

In [8]:
# Allocate the food insecure population and round up to the nearest integer
combined_df['FI_population_2020'] = np.ceil(combined_df['FI Population'] * combined_df['CityWeight']).astype(int)
combined_df.head()

,pop2023,pop2020,growth,name,aland_sqmi,intptlat,intptlong,slug,usps,lng,lat,rank,density,County,TotalCountyPopulation,CityWeight,FI Population,FI_population_2020
0,143776,152846,-0.02019,Jackson,111.086,32.3158,-90.2128,jackson-ms,MS,-90.2128,32.3158,1,1294,Hinds,198680,0.769307,31720,24403
1,26558,27909,-0.01641,Clinton,41.872,32.3544,-90.3419,clinton-ms,MS,-90.3419,32.3544,14,634,Hinds,198680,0.140472,31720,4456
2,12144,12615,-0.01260,Byram,18.364,32.1903,-90.2860,byram-ms,MS,-90.2860,32.1903,32,661,Hinds,198680,0.063494,31720,2015
3,1819,1876,-0.01034,Raymond,2.999,32.2609,-90.4083,raymond-ms,MS,-90.4083,32.2609,105,607,Hinds,198680,0.009442,31720,300
4,1481,1312,0.04149,Terry,3.813,32.1061,-90.2926,terry-ms,MS,-90.2926,32.1061,122,388,Hinds,198680,0.006604,31720,210


In [9]:
# Load the MSDemandLatLong.csv file
ms_demand = pd.read_csv('MSDemandLatLong.csv')
ms_demand['County'] = ms_demand['County'].str.replace(' County', '')
# Merge the "2021 Cost Per Meal" from MSDemandLatLong.csv into the combined_df
combined_df = pd.merge(combined_df, ms_demand[['County', '2021 Cost Per Meal']], on='County')
combined_df.head()


,pop2023,pop2020,growth,name,aland_sqmi,intptlat,intptlong,slug,usps,lng,lat,rank,density,County,TotalCountyPopulation,CityWeight,FI Population,FI_population_2020,2021 Cost Per Meal
0,143776,152846,-0.02019,Jackson,111.086,32.3158,-90.2128,jackson-ms,MS,-90.2128,32.3158,1,1294,Hinds,198680,0.769307,31720,24403,3.77
1,26558,27909,-0.01641,Clinton,41.872,32.3544,-90.3419,clinton-ms,MS,-90.3419,32.3544,14,634,Hinds,198680,0.140472,31720,4456,3.77
2,12144,12615,-0.01260,Byram,18.364,32.1903,-90.2860,byram-ms,MS,-90.2860,32.1903,32,661,Hinds,198680,0.063494,31720,2015,3.77
3,1819,1876,-0.01034,Raymond,2.999,32.2609,-90.4083,raymond-ms,MS,-90.4083,32.2609,105,607,Hinds,198680,0.009442,31720,300,3.77
4,1481,1312,0.04149,Terry,3.813,32.1061,-90.2926,terry-ms,MS,-90.2926,32.1061,122,388,Hinds,198680,0.006604,31720,210,3.77


In [10]:
# Select the relevant columns and rename them
final_df = combined_df[['name', 'County', 'FI_population_2020', '2021 Cost Per Meal', 'lat', 'lng']]
final_df.rename(columns={'name': 'City', 'FI_population_2020': "FI Population 2021", 'lat': 'Latitude', 'lng': 'Longitude'}, inplace=True)
final_df.head()

/var/folders/ct/f6mn147x7cxdht6d71v76t6m0000gp/T/ipykernel_35877/4122148379.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'name': 'City', 'FI_population_2020': "FI Population 2021", 'lat': 'Latitude', 'lng': 'Longitude'}, inplace=True)


,City,County,FI Population 2021,2021 Cost Per Meal,Latitude,Longitude
0,Jackson,Hinds,24403,3.77,32.3158,-90.2128
1,Clinton,Hinds,4456,3.77,32.3544,-90.3419
2,Byram,Hinds,2015,3.77,32.1903,-90.2860
3,Raymond,Hinds,300,3.77,32.2609,-90.4083
4,Terry,Hinds,210,3.77,32.1061,-90.2926


In [11]:
# Save the final DataFrame to a CSV file
final_df.to_csv('MS_City_Demand.csv', index=False)

## Manhattan Distances 

In [12]:
pantry_data = pd.read_csv("MSPantryLatLong.csv")
pantry_data.head()

,Name,Address,Latitude,Longitude
0,Stewpot,"69 E Franklin Street, Natchez, MS",31.556106,-91.389928
1,Rose Hill Baptist Church,"607 Madison Street, Natchez, MS",31.562803,-91.397244
2,Pilgrim Baptist Church,"117 Pilgrim Road, Natchez, MS",31.567402,-91.372177
3,First Assembly of God,"150 Liberty Street, Natchez, MS",31.552891,-91.382085
4,61 South Church of Christ,"6 Beau Pre Road, Natchez, MS",31.482406,-91.368783


In [13]:
from scipy.spatial.distance import cdist
import numpy as np



# Extracting latitudes and longitudes for pantries and cities
pantry_locations = pantry_data[['Latitude', 'Longitude']].to_numpy()
city_locations = final_df[['Latitude', 'Longitude']].to_numpy()

# Function to calculate Manhattan distance in miles
def manhattan_distance(lat_lng1, lat_lng2):
    # Approximate radius of earth in km
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat1, lon1 = np.radians(lat_lng1)
    lat2, lon2 = np.radians(lat_lng2)

    # Difference in coordinates
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Manhattan distance calculation
    a = np.abs(dlat) * R
    b = np.abs(dlon) * R * np.cos((lat1 + lat2) / 2)
    distance = (a + b) * 0.621371  # Convert km to miles

    return distance

# Computing the distance matrix
#distance_matrix = cdist(pantry_locations, city_locations, metric=manhattan_distance)

# Creating a DataFrame for the distance matrix
#distance_df = pd.DataFrame(distance_matrix, index=pantry_data['Name'], columns=final_df['City'])

#distance_df.head()





In [14]:
# Save the DataFrame to a CSV file
csv_file_path = 'PantryToCityDistanceMatrix.csv'
distance_df.to_csv(csv_file_path)

NameError: name 'distance_df' is not defined